# Prepare Sankey Data

In [ ]:
from tqdm.notebook import tqdm
from concurrent import futures

import itertools
import pandas as pd
import numpy as np

In [ ]:
subset = pd.read_csv('alum-edu-hist.csv')
subset

In [ ]:
subset.columns

In [ ]:
all_majors = pd.concat((subset['Major #1'], subset['Major #2'])).unique()
all_minors = pd.concat((subset['Minor #1'], subset['Minor #2'])).unique()

In [ ]:
def minor_search(major1, major2, major_subset):
    tabs = []    
    for minor1 in all_minors:
        if minor1 is np.nan:
            cnt = major_subset[(major_subset['Minor #1'].isna()) & (major_subset['Minor #2'].isna())].shape[0]
            if cnt:
                tabs.append({
                'major1': major1,
                'major2': 'None' if isinstance(major2, float) else major2,
                'minor1': 'None',
                'minor2': 'None',
                'cnt': cnt,
                })
                continue
        if minor1 in (major1, major2):
            continue
            
        for minor2 in all_minors:
            if minor2 is np.nan:
                if minor2 in (major1, major2):
                    continue
                
                cnt = major_subset[(major_subset['Minor #1'] == minor1) & (major_subset['Minor #2'].isna())].shape[0]
            else:
                cnt = major_subset[((major_subset['Minor #1'] == minor1) & (major_subset['Minor #2'] == minor2)) | \
                                   ((major_subset['Minor #1'] == minor2) & (major_subset['Minor #2'] == minor1))].shape[0]
                    
            if cnt:
                tabs.append({
                    'major1': major1,
                    'major2': 'None' if isinstance(major2, float) else major2,
                    'minor1': minor1,
                    'minor2': 'None' if minor2 is np.nan else minor2,
                    'cnt': cnt
                })
    return tabs

In [ ]:
cross_tab = []
for major1 in tqdm(all_majors, desc='Major 1'):
    if major1 is np.nan:
        continue
        
    major2_subsets = []
    for major2 in all_majors:
        if major2 is np.nan:
            major_specific = (subset['Major #1'] == major1) & (subset['Major #2'].isna())
        else:
            major_specific = ((subset['Major #1'] == major1) & (subset['Major #2'] == major2)) | \
                             ((subset['Major #1'] == major2) & (subset['Major #2'] == major1))
        major2_subsets.append(subset[major_specific])
    
    with futures.ProcessPoolExecutor() as executor:
        cross_tab.append(list(tqdm(executor.map(minor_search, itertools.repeat(major1), all_majors, major2_subsets),
                                   total=len(all_majors), desc='Major 2', leave=False)))